In [318]:
## Run selenium and chrome driver to scrape data from cloudbytes.dev
import time
import os.path
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import pandas as pd

In [319]:
## Setup chrome options
chrome_options = Options()
chrome_options.headless = True # Ensure GUI is off
chrome_options.add_argument("--window-size=1920,1200")

In [320]:
# Set path to chromedriver as per your configuration
homedir = os.path.expanduser("~")
webdriver_service = Service(f"{homedir}/ao3lockwood-co/chromedriver")

In [321]:
# Choose Chrome Browser
browser = webdriver.Chrome(service=webdriver_service, options=chrome_options)

In [322]:
# Get page
pagenum=1
link="https://archiveofourown.org/tags/Lockwood%20*a*%20Co*d*%20-%20Jonathan%20Stroud/works?page="+str(pagenum)
browser.get(link)
maxpagenum=browser.find_element(By.XPATH,'//ol[1]/li[13]').text.strip()

In [323]:
def get_data(browser):
    # Find all the fanfic works on the page
    works = browser.find_elements(By.XPATH, '//ol[2]/li')
    # Iterate through each work and extract author and datetime
    data=[]
    for work in works:
        h4 = work.find_element(By.TAG_NAME,'h4')
        a = h4.find_elements(By.TAG_NAME, 'a')
        title = a[0].text.strip()
        author = a[1].text.strip()
        datetime = work.find_element(By.CLASS_NAME, 'datetime').text.strip()
        warning=work.find_element(By.TAG_NAME,'strong').text.strip()
        relationship_list = []
        relationship = work.find_elements(By.CLASS_NAME,'relationships')
        for r in relationship:
            relationship_list.append(r.text.strip())
        character_list = []
        characters = work.find_elements(By.CLASS_NAME, 'characters')
        for char in characters:
            character_list.append(char.text.strip())
        tags_list = []
        tags = work.find_elements(By.CLASS_NAME,'freeforms')
        for tag in tags:
            tags_list.append(tag.text.strip())
        summary = work.find_element(By.TAG_NAME, 'blockquote').text.strip()
        language = work.find_elements(By.CLASS_NAME,'language')[1].text.strip()
        words = work.find_elements(By.CLASS_NAME,'words')[1].text.strip()
        chapters = work.find_elements(By.CLASS_NAME,'chapters')[1].text.strip()
        kudos = work.find_elements(By.CLASS_NAME,'kudos')[1].text.strip()
        hits = work.find_elements(By.CLASS_NAME,'hits')[1].text.strip()
        link = a[0].get_attribute("href")
        data.append({'title':title, 'author':author, 'dateupdated':datetime, 'link':link, 'warning':warning,'relationship':relationship_list,
                     'characters':character_list,'tags':tags_list, 'summary':summary, 'language':language,'words':words,
                     'chapters':chapters, 'kudos':kudos, 'hits':hits})
    return pd.DataFrame(data)
    

In [324]:
working_df=get_data(browser)
print(working_df)

                               title              author  dateupdated  \
0                          Speak Now        SnazzyCyborg  18 Apr 2023   
1                  quiet like a fire        RainShadow07  17 Apr 2023   
2                        My Rosemary             SinSong  18 Apr 2023   
3                    You've Got Mail       spinnaker1509  18 Apr 2023   
4   will i stay numb or regain love?  Fandomscraziness22  17 Apr 2023   
5               Rest Your Weary Head              mliz18  18 Apr 2023   
6                                                                       
7                                                                       
8                                                                       
9                                                                       
10                                                                      
11                                                                      
12                                                 

In [325]:
#Wait for 10 seconds
time.sleep(10)
browser.quit()